In [ ]:


from tensorflow.keras.applications import MobileNet

model = MobileNet(weights="imagenet")  # Load pre-trained MobileNet
model.save("mobilenet_model.keras")       # Save as a .h5 file


In [ ]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model("mobilenet_model.h5")

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the .tflite file
with open("mobilenet_model.tflite", "wb") as f:
    f.write(tflite_model)

In [20]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="mobilenet_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print model input and output details for verification
print("Input details:", input_details)
print("Output details:", output_details)

# Prepare an input image
image = Image.open("meat.jpg").convert("RGB").resize((224, 224))  # Ensure RGB and resize
input_data = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1]
input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension

# Verify input shape matches model's expectation
print("Input shape (expected):", input_details[0]['shape'])
print("Input shape (provided):", input_data.shape)

# Run inference
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Get predictions
output_data = interpreter.get_tensor(output_details[0]['index'])

# Load class labels and find the most likely class
with open("labels.txt", "r") as f:
    labels = f.readlines()

predicted_index = np.argmax(output_data[0])  # Index of the highest probability
predicted_label = labels[predicted_index].strip()

# Print the predicted label
print(f"Predicted Label: {predicted_label}")

Input details: [{'name': 'serving_default_input_layer_2:0', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 94, 'shape': array([   1, 1000], dtype=int32), 'shape_signature': array([  -1, 1000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Input shape (expected): [  1 224 224   3]
Input shape (provided): (1, 224, 224, 3)
Predicted Label: 930: 'French loaf',
